In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Сиды

In [1]:
import os
import random
import numpy as np
import torch

try:
    # Hugging Face convenience fn; sets Python/Rand, NumPy, Torch seeds
    from transformers import set_seed  
except ImportError:
    set_seed = None

def seed_everything(seed: int = 42):
    # 1. Python built‑ins
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    # 2. NumPy
    np.random.seed(seed)

    # 3. PyTorch (CPU & GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 4. CuDNN: make deterministic, but may slow you down
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 5. Transformers (if installed)
    if set_seed is not None:
        set_seed(seed)

# call it!
seed_everything(42)

2025-07-29 10:47:38.968224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753786059.201253      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753786059.270242      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Импорты

In [2]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments

In [3]:
import torch
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import pandas as pd

# Загрузка данных

In [4]:
def read_texts_from_dir(dir_path):
  """
  Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

  Params:
    dir_path (str): path to the directory with data
  """
  # Count number of directories in the provided path
  dir_count = sum(os.path.isdir(os.path.join(root, d)) for root, dirs, _ in os.walk(dir_path) for d in dirs)
  data=[0 for _ in range(dir_count)]
  print(f"Number of directories: {dir_count}")

  # For each directory, read both file_1.txt and file_2.txt and save results to the list
  i=0
  for folder_name in sorted(os.listdir(dir_path)):
    folder_path = os.path.join(dir_path, folder_name)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, 'file_1.txt'), 'r', encoding='utf-8') as f1:
          text1 = f1.read().strip()
        with open(os.path.join(folder_path, 'file_2.txt'), 'r', encoding='utf-8') as f2:
          text2 = f2.read().strip()
        index = int(folder_name[-4:])
        data[i]=(index, text1, text2)
        i+=1
      except Exception as e:
        print(f"Error reading directory {folder_name}: {e}")

  # Change list with results into pandas DataFrame
  df = pd.DataFrame(data, columns=['id', 'file_1', 'file_2']).set_index('id')
  return df

In [5]:
train_path="/kaggle/input/fake-or-real-the-impostor-hunt/data/train"
train=read_texts_from_dir(train_path)
test_path="/kaggle/input/fake-or-real-the-impostor-hunt/data/test"
test=read_texts_from_dir(test_path)

Number of directories: 95
Number of directories: 1068


In [ ]:
train_labels=pd.read_csv('/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv')
#train_repository='/kaggle/input/fake-or-real-the-impostor-hunt/data/train/'
#test_repository='/kaggle/input/fake-or-real-the-impostor-hunt/data/test/'

In [6]:
train_labels = pd.read_csv('/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv')
train = (
    train.reset_index()                 # bring 'id' back as a column
         .merge(train_labels, on='id', how='inner')
         .set_index('id')
)

# sanity check
assert train['real_text_id'].notna().all(), "Labels have NaNs after merge."


In [ ]:
train = train.join(train_labels)

# Чистка текста

In [ ]:
import re 

In [ ]:
#def clean():
    

In [7]:
test_size = 0.1
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=test_size, stratify=train['real_text_id'], random_state=42)

# Датасет

In [ ]:
#class ImposterDataset(Dataset):
#    def __init__(self, ):
#        
#    def __len__(self):
#        
#    def __getitem__(self, idx):
        

In [ ]:
class ImposterDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.texts1 = df['file_1'].tolist()
        self.texts2 = df['file_2'].tolist()
        self.labels = df['real_text_id'].tolist() if 'real_text_id' in df.columns else None
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts1)

    def __getitem__(self, idx):
        # Prepare texts
        text1 = self.texts1[idx]
        text2 = self.texts2[idx]
        encoding = self.tokenizer(
            text1,
            text2,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        if self.labels is not None:
            lbl_orig = self.labels[idx]
            # lbl_orig in {1,2} -> shift to 0/1 for model
            assert lbl_orig in [1,2], f'Unexpected label {lbl_orig}, expected 1 or 2'
            lbl = lbl_orig - 1
            #item['labels'] = torch.tensor(lbl, dtype=torch.float)
            item['labels'] = torch.tensor([lbl], dtype=torch.float)
        return item


# Модель

candidate_models = [
    'intfloat/e5-small-v2',
    'distilbert-base-uncased',
    'roberta-base',
    'sentence-transformers/all-MiniLM-L6-v2'
]

In [ ]:
model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name # разрешаем пересоздать слой
)

In [ ]:
#model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Токенайзер

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_dataset = ImposterDataset(train_df, tokenizer)
val_dataset   = ImposterDataset(val_df,   tokenizer)
test_dataset  = ImposterDataset(test.reset_index(drop=True), tokenizer)

# Метрика

In [ ]:
'''def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.squeeze()  # (N,)
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)
    from sklearn.metrics import accuracy_score, f1_score
    return {'accuracy': accuracy_score(labels, preds),
            'f1': f1_score(labels, preds)}'''

In [ ]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # preds shape: (N, 1) or (N,)
    logits = preds.squeeze()
    probs = 1 / (1 + np.exp(-logits))
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)
    from sklearn.metrics import accuracy_score, f1_score
    return {'accuracy': accuracy_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)}

# Обучение

## Training arguments

In [ ]:
args=TrainingArguments(
    output_dir='gol',
    num_train_epochs=10,
    learning_rate=3e-5,
    report_to='none',
    lr_scheduler_type='linear',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    
    eval_strategy='steps',
    eval_steps=10,
    warmup_ratio=0.1015,
    save_strategy='epoch',
    greater_is_better=True,
    

    
    
)

In [ ]:
args = TrainingArguments(
    output_dir='gol',
    num_train_epochs=10,
    #learning_rate=5e-6,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy='steps',
    eval_steps=20,
    logging_dir='logs',
    logging_steps=20,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    report_to='none',
)

## Trainer

In [ ]:
trainer=Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    args=args,

    
)

In [ ]:
trainer.train()

In [ ]:
# ============================
# Pairwise Ranking: setup
# ============================
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "microsoft/deberta-v3-base"   # good default; switch to -large if VRAM allows
MAX_LEN = 384                               # tweak (384–512). You can auto-tune by length quantile if needed.

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ----------------------------
# Dataset: returns paired inputs and a label in {+1, -1}
# swap_prob adds order augmentation during training
# ----------------------------
class PairwiseDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=384, with_labels=True, swap_prob=0.5):
        self.text1 = df["file_1"].tolist()
        self.text2 = df["file_2"].tolist()
        self.with_labels = with_labels
        self.swap_prob = swap_prob if with_labels else 0.0
        if with_labels:
            # label: +1 if file_1 is real, -1 if file_2 is real
            y = (df["real_text_id"].values == 1).astype(np.int64)
            self.labels = torch.from_numpy(2*y - 1).float()  # {0,1} -> {-1,+1}
        else:
            self.labels = None
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self): 
        return len(self.text1)

    def _encode(self, t):
        enc = self.tokenizer(
            t,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {k: v.squeeze(0) for k, v in enc.items()}

    def __getitem__(self, i):
        t1, t2 = self.text1[i], self.text2[i]
        label = None if self.labels is None else self.labels[i]

        # Randomly swap order during training (label flips sign)
        if self.swap_prob > 0 and np.random.rand() < self.swap_prob:
            t1, t2 = t2, t1
            if label is not None:
                label = -label

        item = {}
        enc1 = self._encode(t1)
        enc2 = self._encode(t2)
        # we namespace keys: *1 and *2
        for k, v in enc1.items():
            item[f"{k}1"] = v
        for k, v in enc2.items():
            item[f"{k}2"] = v
        if label is not None:
            item["labels"] = label
        return item

# ----------------------------
# Model: wraps a single-text scorer (num_labels=1) and computes pairwise loss
# ----------------------------
class PairwiseRanker(nn.Module):
    def __init__(self, base_model_name):
        super().__init__()
        # Single‑logit scorer
        self.scorer = AutoModelForSequenceClassification.from_pretrained(
            base_model_name, num_labels=1
        )
        self.loss_fn = nn.MarginRankingLoss(margin=0.2)

    def forward(self, **batch):
        # split inputs for side 1 and 2
        def gather(prefix):
            out = {}
            for k in list(batch.keys()):
                if k.endswith(prefix):
                    out[k[:-1]] = batch[k]   # strip trailing '1' or '2'
            return out

        b1 = gather("1")  # input_ids1 -> input_ids, attention_mask1 -> attention_mask, etc.
        b2 = gather("2")

        # Some tokenizers don't provide token_type_ids; make sure we don't pass missing keys
        # (Hugging Face handles absent keys fine, so no extra work needed)

        s1 = self.scorer(**b1).logits.squeeze(-1)  # (B,)
        s2 = self.scorer(**b2).logits.squeeze(-1)  # (B,)

        logits_diff = (s1 - s2)                    # (B,)

        if "labels" in batch and batch["labels"] is not None:
            # labels in {+1, -1}
            y = batch["labels"].view_as(logits_diff)
            loss = self.loss_fn(s1, s2, y)
            return {"loss": loss, "logits": logits_diff}

        return {"logits": logits_diff}

# ----------------------------
# Metrics: pairwise accuracy
# ----------------------------
def compute_metrics(eval_pred):
    # Trainer passes predictions (logits) and labels
    preds = eval_pred.predictions.squeeze()
    labels = eval_pred.label_ids.squeeze()
    # correct if sign matches (positive -> text1 real)
    acc = ( (preds > 0) == (labels > 0) ).mean().item()
    return {"pairwise_accuracy": acc}

# ============================
# Build datasets
# ============================
train_dataset = PairwiseDataset(train_df, tokenizer, max_length=MAX_LEN, with_labels=True,  swap_prob=0.5)
val_dataset   = PairwiseDataset(val_df,   tokenizer, max_length=MAX_LEN, with_labels=True,  swap_prob=0.0)
test_dataset  = PairwiseDataset(test,     tokenizer, max_length=MAX_LEN, with_labels=False, swap_prob=0.0)

# ============================
# Trainer & training
# ============================
args = TrainingArguments(
    output_dir="pair_rank_gol",
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.05,
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="pairwise_accuracy",
    greater_is_better=True,
    report_to="none",
    gradient_accumulation_steps=1,
)

model = PairwiseRanker(MODEL_NAME)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# ============================
# Validation sanity check (pairwise accuracy)
# ============================
val_out = trainer.predict(val_dataset)
val_logits = val_out.predictions.squeeze()
val_preds = (val_logits > 0).astype(int)        # 1 -> file_1 real, 0 -> file_2 real
val_true  = (val_out.label_ids.squeeze() > 0).astype(int)
val_pair_acc = (val_preds == val_true).mean()
print(f"Validation pairwise accuracy: {val_pair_acc:.4f}")

# ============================
# Inference & submission
# ============================
test_out = trainer.predict(test_dataset)
logits_diff = test_out.predictions.squeeze()
which_real = np.where(logits_diff > 0, 1, 2)   # sign decides which side is real

submission = pd.DataFrame({
    "id": test.index, 
    "real_text_id": which_real
})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")


In [ ]:
# 1) Flatten pairs into single-text rows
def make_rowwise(df):  # df has columns: file_1, file_2, real_text_id, index 'id'
    rows = []
    for pid, row in df.reset_index().iterrows():
        rows.append({'pair_id': row['id'], 'text': row['file_1'], 'label': int(row['real_text_id']==1)})
        rows.append({'pair_id': row['id'], 'text': row['file_2'], 'label': int(row['real_text_id']==2)})
    return pd.DataFrame(rows)

train_row = make_rowwise(train_df)
val_row   = make_rowwise(val_df)

# 2) Dataset for single text
class RealnessDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512, with_labels=True):
        self.texts  = df['text'].tolist()
        self.labels = df['label'].tolist() if with_labels else None
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = self.tokenizer(self.texts[i], truncation=True, padding='max_length',
                             max_length=self.max_length, return_tensors='pt')
        item = {k: v.squeeze(0) for k,v in enc.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[i], dtype=torch.long)
        return item

# 3) Model: 2-class head
#model_name = 'microsoft/deberta-v3-base'  # or 'roberta-base' deberta-v3-large
model_name = 'microsoft/deberta-v3-base'
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_ds = RealnessDataset(train_row, tokenizer)
val_ds   = RealnessDataset(val_row,   tokenizer)

# 4) Metrics aligned to Kaggle (accuracy)
def compute_metrics(p):
    from sklearn.metrics import accuracy_score, f1_score
    y_pred = p.predictions.argmax(-1)
    y_true = p.label_ids
    return {"accuracy": accuracy_score(y_true, y_pred),
            "f1": f1_score(y_true, y_pred)}

args = TrainingArguments(
    output_dir='gol',
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy='steps',
    eval_steps=200,
    logging_steps=200,
    save_strategy='steps',
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    report_to='none',
)

trainer = Trainer(model=model, tokenizer=tokenizer,
                  train_dataset=train_ds, eval_dataset=val_ds,
                  compute_metrics=compute_metrics, args=args)
trainer.train()

# 5) Inference: score both texts, choose the higher "real" prob within each pair
# build test as two rows per pair
def make_rowwise_test(df):
    rows=[]
    for pid, row in df.reset_index().iterrows():
        rows.append({'pair_id': row['id'], 'which':1, 'text': row['file_1']})
        rows.append({'pair_id': row['id'], 'which':2, 'text': row['file_2']})
    return pd.DataFrame(rows)

test_row = make_rowwise_test(test)
test_ds  = RealnessDataset(test_row, tokenizer, with_labels=False)
out = trainer.predict(test_ds)

import numpy as np, pandas as pd
probs = np.exp(out.predictions) / np.exp(out.predictions).sum(-1, keepdims=True)
test_row = test_row.assign(p_real=probs[:,1])
choice = (test_row.sort_values(['pair_id','which'])
                 .groupby('pair_id', as_index=False)
                 .apply(lambda g: pd.Series({'real_text_id': int(g.loc[g['p_real'].idxmax(), 'which'])})))
submission = choice.rename(columns={'pair_id':'id'})[['id','real_text_id']]
submission.to_csv('submission.csv', index=False)


In [9]:
from dataclasses import dataclass
from typing import Dict, List, Any
import torch

@dataclass
class PairwiseCollator:
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        batch: Dict[str, torch.Tensor] = {}
        keys = features[0].keys()
        for k in keys:
            vals = [f[k] for f in features]
            if isinstance(vals[0], torch.Tensor):
                batch[k] = torch.stack(vals)
            else:
                batch[k] = torch.tensor(vals)
        return batch


In [10]:
# ============================
# Pairwise Ranking: setup
# ============================
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#MODEL_NAME = "microsoft/deberta-v3-base"
MODEL_NAME = "google/electra-large-discriminator"
# good default; switch to -large if VRAM allows
MAX_LEN = 384                               # tweak (384–512). You can auto-tune by length quantile if needed.

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ----------------------------
# Dataset: returns paired inputs and a label in {+1, -1}
# swap_prob adds order augmentation during training
# ----------------------------
class PairwiseDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=384, with_labels=True, swap_prob=0.5):
        self.text1 = df["file_1"].tolist()
        self.text2 = df["file_2"].tolist()
        self.with_labels = with_labels
        self.swap_prob = swap_prob if with_labels else 0.0
        if with_labels:
            # label: +1 if file_1 is real, -1 if file_2 is real
            y = (df["real_text_id"].values == 1).astype(np.int64)
            self.labels = torch.from_numpy(2*y - 1).float()  # {0,1} -> {-1,+1}
        else:
            self.labels = None
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self): 
        return len(self.text1)

    def _encode(self, t):
        enc = self.tokenizer(
            t,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {k: v.squeeze(0) for k, v in enc.items()}

    def __getitem__(self, i):
        t1, t2 = self.text1[i], self.text2[i]
        label = None if self.labels is None else self.labels[i]

        # Randomly swap order during training (label flips sign)
        if self.swap_prob > 0 and np.random.rand() < self.swap_prob:
            t1, t2 = t2, t1
            if label is not None:
                label = -label

        item = {}
        enc1 = self._encode(t1)
        enc2 = self._encode(t2)
        # we namespace keys: *1 and *2
        for k, v in enc1.items():
            item[f"{k}1"] = v
        for k, v in enc2.items():
            item[f"{k}2"] = v
        if label is not None:
            item["labels"] = label
        return item

# ----------------------------
# Model: wraps a single-text scorer (num_labels=1) and computes pairwise loss
# ----------------------------
class PairwiseRanker(nn.Module):
    def __init__(self, base_model_name):
        super().__init__()
        # Single‑logit scorer
        self.scorer = AutoModelForSequenceClassification.from_pretrained(
            base_model_name, num_labels=1
        )
        self.loss_fn = nn.MarginRankingLoss(margin=0.2)

    def forward(self, **batch):
        # split inputs for side 1 and 2
        def gather(prefix):
            out = {}
            for k in list(batch.keys()):
                if k.endswith(prefix):
                    out[k[:-1]] = batch[k]   # strip trailing '1' or '2'
            return out

        b1 = gather("1")  # input_ids1 -> input_ids, attention_mask1 -> attention_mask, etc.
        b2 = gather("2")

        # Some tokenizers don't provide token_type_ids; make sure we don't pass missing keys
        # (Hugging Face handles absent keys fine, so no extra work needed)

        s1 = self.scorer(**b1).logits.squeeze(-1)  # (B,)
        s2 = self.scorer(**b2).logits.squeeze(-1)  # (B,)

        logits_diff = (s1 - s2)                    # (B,)

        if "labels" in batch and batch["labels"] is not None:
            # labels in {+1, -1}
            y = batch["labels"].view_as(logits_diff)
            loss = self.loss_fn(s1, s2, y)
            return {"loss": loss, "logits": logits_diff}

        return {"logits": logits_diff}

# ----------------------------
# Metrics: pairwise accuracy
# ----------------------------
def compute_metrics(eval_pred):
    # Trainer passes predictions (logits) and labels
    preds = eval_pred.predictions.squeeze()
    labels = eval_pred.label_ids.squeeze()
    # correct if sign matches (positive -> text1 real)
    acc = ( (preds > 0) == (labels > 0) ).mean().item()
    return {"pairwise_accuracy": acc}

# ============================
# Build datasets
# ============================
train_dataset = PairwiseDataset(train_df, tokenizer, max_length=MAX_LEN, with_labels=True,  swap_prob=0.5)
val_dataset   = PairwiseDataset(val_df,   tokenizer, max_length=MAX_LEN, with_labels=True,  swap_prob=0.0)
test_dataset  = PairwiseDataset(test,     tokenizer, max_length=MAX_LEN, with_labels=False, swap_prob=0.0)

# ============================
# Trainer & training
# ============================
args = TrainingArguments(
    output_dir="pair_rank_gol",
    num_train_epochs=6,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.05,
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="pairwise_accuracy",
    greater_is_better=True,
    report_to="none",
    gradient_accumulation_steps=1,
    remove_unused_columns=False,   # <-- keep custom keys
)
model = PairwiseRanker(MODEL_NAME)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=PairwiseCollator(),  # <-- custom collator
)



trainer.train()

# ============================
# Validation sanity check (pairwise accuracy)
# ============================
val_out = trainer.predict(val_dataset)
val_logits = val_out.predictions.squeeze()
val_preds = (val_logits > 0).astype(int)        # 1 -> file_1 real, 0 -> file_2 real
val_true  = (val_out.label_ids.squeeze() > 0).astype(int)
val_pair_acc = (val_preds == val_true).mean()
print(f"Validation pairwise accuracy: {val_pair_acc:.4f}")

# ============================
# Inference & submission
# ============================
test_out = trainer.predict(test_dataset)
logits_diff = test_out.predictions.squeeze()
which_real = np.where(logits_diff > 0, 1, 2)   # sign decides which side is real

submission = pd.DataFrame({
    "id": test.index, 
    "real_text_id": which_real
})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_36/215824418.py:155: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 29.12 MiB is free. Process 2375 has 15.86 GiB memory in use. Of the allocated memory 15.56 GiB is allocated by PyTorch, and 7.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
test_out = trainer.predict(test_dataset)
logits_diff = test_out.predictions.squeeze()
which_real = np.where(logits_diff > 0, 1, 2)   # sign decides which side is real

submission = pd.DataFrame({
    "id": test.index, 
    "real_text_id": which_real
})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")

In [ ]:
submission

# Submission

In [ ]:
preds_out = trainer.predict(test_dataset)
logits = preds_out.predictions.squeeze()
probs  = 1 / (1 + np.exp(-logits))
preds  = (probs > 0.5).astype(int)
submission = pd.DataFrame({'id': test.index, 'label': preds + 1})

In [ ]:
'''preds_out = trainer.predict(test_dataset)
preds     = np.argmax(preds_out.predictions, axis=1)
submission = pd.DataFrame({'id': test.index.tolist(), 'label': preds + 1})  # shift back to {1,2}
submission.to_csv('submission.csv', index=False)
print('Saved submission.csv')'''

In [ ]:
best_thr=0.5

In [ ]:
logits = preds_out.predictions.squeeze()          # (N,)
probs  = 1 / (1 + np.exp(-logits))                # sigmoid
preds  = (probs > best_thr).astype(int)           # 0/1
submission = pd.DataFrame({'id': test.index, 'label': preds + 1})

In [ ]:
submission.to_csv('eeeee.csv', index=False)

In [ ]:
submission

In [ ]:
def inference_and_submission(trainer, test_dataset, test_index):
    preds_out = trainer.predict(test_dataset)
    preds     = np.argmax(preds_out.predictions, axis=1)
    return pd.DataFrame({'id': test_index, 'label': preds + 1})  # shift back to {1,2}

In [ ]:
from tqdm.auto import tqdm

def evaluate_models(model_names, train_df, val_df, tokenizer_class, model_class):
    results = []
    for model_name in model_names:
        print(f"=== Evaluating {model_name} ===")
        tok = tokenizer_class.from_pretrained(model_name)
        mdl = model_class.from_pretrained(model_name, num_labels=2)
        # Freeze encoder
        for n,p in mdl.named_parameters():
            if 'classifier' not in n:
                p.requires_grad = False
        train_ds = ImposterDataset(train_df, tok)
        val_ds   = ImposterDataset(val_df,   tok)
        args = TrainingArguments(
            output_dir=f'gol_{model_name.replace("/","_")}',
            num_train_epochs=5,
            learning_rate=5e-6,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            weight_decay=0.01,
            eval_strategy='epoch',
            save_strategy='no',
            logging_strategy='no',
            load_best_model_at_end=False
        )
        tr = Trainer(
            model=mdl,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            tokenizer=tok,
            
        )
        tr.train()
        metrics = tr.evaluate()
        results.append({'model': model_name, 'eval_accuracy': metrics['eval_accuracy'], 'eval_f1': metrics['eval_f1']})
    return pd.DataFrame(results)

# Define candidate models
candidate_models = [
    'intfloat/e5-small-v2',
    'distilbert-base-uncased',
    'roberta-base',
    'sentence-transformers/all-MiniLM-L6-v2'
]

# Run evaluation
comparison_df = evaluate_models(
    candidate_models,
    train_df, val_df,
    
    AutoTokenizer, AutoModelForSequenceClassification
)


print(comparison_df)

# Pick best model and retrain full pipeline if desired
best_model = comparison_df.sort_values('eval_f1', ascending=False).iloc[0]['model']

